## Set Up

In [5]:
import arcpy
from datetime import datetime
import os
import sys
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import sqlalchemy as sa
import pandas as pd
from arcgis import GIS
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor, FeatureLayer
import pandas as pd
import numpy as np
import requests

gis = GIS()
# # Set Pandas display options to show all rows and columns
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# in memory output file path
wk_memory = "memory" + "\\"

# set workspace and sde connections 
working_folder = "F:/GIS/GIS_DATA/Monitoring/"
workspace ='F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'
arcpy.env.workspace = 'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'

# network path to connection files
filePath = r'F:\Research and Analysis\Workspace\Sarah'

# database file path 
sdeBase    = os.path.join(filePath, "SarahVector.sde")
#sdeCollect = os.path.join(filePath, "Collection.sde")

#SDE Collection most our our files are in the SDE.Survey folder
#Collection.sde.SDE.Survey


# local variables
fdata = os.path.join(sdeBase, "sde.SDE.Monitoring")
## Final feature class to append to in Enterprise Geodatabase double check this
sdeSEZ = os.path.join(sdeBase, "sde.SDE.Monitoring\sde.SDE.SEZ_Assessment_Unit")

## GDB with Raw Data straight from S123 not in the original folder (that one is not edited)
headcut23gdb = os.path.join(working_folder,"Stream_Headcut","Stream_Headcut_Survey", "Stream_Headcut_Survey_2023.gdb")
headcut22gdb = os.path.join(working_folder,"Stream_Headcut","Stream_Headcut_Survey", "Stream_Headcut_Survey_2022.gdb")
headcut20gdb = os.path.join(working_folder,"Stream_Headcut","Stream_Headcut_Survey", "Stream_Headcut_Survey_2020.gdb")
erosion23gdb = os.path.join(working_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2023.gdb")
erosion22gdb = os.path.join(working_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2022.gdb")
erosion20gdb = os.path.join(working_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2020.gdb")
channelincision23gdb = os.path.join(working_folder,"Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2023.gdb")
channelincision22gdb = os.path.join(working_folder, "Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2022.gdb")
channelincision20gdb = os.path.join(working_folder, "Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2020.gdb")
invasiveplant23gdb= os.path.join(working_folder,"Invasive_Species","Invasive_Species_Survey","Invasive_Species_Survey_2023.gdb")
invasiveplant22gdb= os.path.join(working_folder,"Invasive_Species","Invasive_Species_Survey","Invasive_Species_Survey_2022.gdb")
invasiveplant20gdb= os.path.join(working_folder,"Invasive_Species","Invasive_Species_Survey","Invasive_Species_Survey_2020.gdb")

#This is thelocatoin for the final SEZ GDB to be updated in the gdb on f drive in the AssessmentUnits Master (polygon) i believe
#FinalGDBtoupdate:F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data \SEZ_Data.gdb

#Monitoring Dashboard location
#Finalsdelocation:f'Vector.SDE' Sde.Monitoring Sde. SEZ_Assessment_Unit

#Threshold Location? sde.tabular

### Stream Erosion 

In [ ]:
#one gdb at a time ?

#2023gdb
# Name of the feature class or table
fc_name = 'Stream_Erosion'

# List to store rows from the attribute table
data = []

# Fields to include in the DataFrame
fields22 = ['Assessment_Unit_Name', 'Bank_Type', 'Shape_Length']  # Add the names of the fields you want to include

# Iterate over the attribute table using SearchCursor
with arcpy.da.SearchCursor(f'{erosion23gdb}\\{fc_name}', fields) as cursor:
    for row in cursor:
        data.append(row)

# Create a DataFrame from the list of rows
erosion23df = pd.DataFrame(data, columns=fields)

# Print the DataFrame
print(erosion23df)

#2022gdb
# Name of the feature class or table
fc22_name = 'Stream_Erosion_2022'

# List to store rows from the attribute table
data22 = []

# Iterate over the attribute table using SearchCursor
with arcpy.da.SearchCursor(f'{erosion22gdb}\\{fc22_name}', fields22) as cursor:
    for row in cursor:
        data.append(row)

# Create a DataFrame from the list of rows
erosion22df = pd.DataFrame(data22, columns=fields)

# Print the DataFrame
print(erosion22df)

#2020gdb
# Name of the feature class or table
fc20_name = 'Stream_Erosion_Final_2022'

# List to store rows from the attribute table
data20 = []

# Iterate over the attribute table using SearchCursor
with arcpy.da.SearchCursor(f'{erosion20gdb}\\{fc20_name}', fields) as cursor:
    for row in cursor:
        data20.append(row)

# Create a DataFrame from the list of rows
erosion20df = pd.DataFrame(data, columns=fields)

# Print the DataFrame
print(erosion20df)



In [12]:
# Stream Erosion Data Processing from GDB Bank Stability to get this to update sde i will need to pull from 2020-2023 gdbs

# Set the workspace to your Geodatabases
#arcpy.env.workspace = Scratch.Gdbgdb

# List of paths to geodatabases
Erosiongdbs = ['erosion20gdb', 'erosion22gdb', 'erosion23gdb']

# Initialize an empty list to store DataFrames
erosiondfs = []
 # Iterate over features using SearchCursor
    for fc in feature_classes:
    fc_attributes = []
    with arcpy.da.SearchCursor(fc, '*') as cursor:
        for row in cursor:
            fc_attributes.append(row)
    df = pd.DataFrame(fc_attributes)
    erosiondfs.append(df)
with arcpy.da.SearchCursor(erosiondfs, '*') as cursor:
            for row in cursor:
                # Append feature attributes to the list
                erosiondfs.append(row)
        
        # Convert feature attributes to DataFrame
        df = pd.DataFrame(erosiondfs)
# Iterate over each geodatabase
#for gdb in Erosiongdbs:
# List feature classes or tables in the geodatabase
feature_classes = arcpy.ListFeatureClasses("*", "All", gdb)
    
    
    # Iterate over feature classes and tables
  #  for fc in feature_classes:
        # Convert feature class or table to DataFrame
   #     df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(fc, '*'))arcpy.da.SearchCursor()
erosiondfs.append(df)
#
# Concatenate all DataFrames into a single DataFrame
erosion_df = pd.concat(erosiondfs)

# Now you have a single DataFrame containing data from multiple geodatabases

# Input feature class or table its not a feature class? or is it a feature class within the gdb.. Do i need to bring in the table with photos too?
input_fc = 'Stream_Erosion'

# Convert feature class to Pandas DataFrame
erosionfields = ['Assessment_Unit_Name', 'Shape_Length', 'Bank_Type', 'Percent_Unstable',]
erosiondf = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(input_fc, erosionfields), columns=erosionfields)

# Initialize variables
erosion_df['bank_multiplier'] = erosion_df['Bank_Type'].apply(lambda x: 2 if x == 'Both Banks' else (1 if x == 'One Bank' else 0))

# Calculate the total length of banks assessed
total_banks_assessed = (erosion_df['Shape_Length'].sum()) * 2

# Calculate the total length of eroded banks
total_eroded_banks = (erosion_df['Shape_Length'] * erosion_df['bank_multiplier']).sum()

# calculate percent unstable 'percent_unstable' column!!! Bank Stability and we will need to add a column to the dataframe
erosion_df['percent_unstable'] = (total_eroded_banks / total_banks_assessed) * 100

print("Total Banks Assessed:", total_banks_assessed)
print("Total Eroded Banks:", total_eroded_banks)
print("Updated DataFrame:")
print(erosion_df)

#field mapping? what is the fi nal location
#field_mapping ={

    #'Assessment_Unit_Name': 
    #'Shape_Length'
    #"Bank_Type"
    #'Percent_Unstable'
#}

# Create a new DataFrame Final_df with columns based on the field mapping
#df = df.rename(columns=field_mapping)

# Convert DataFrame to Spatially Enabled DataFrame
sedf = GeoAccessor.from_xy(erosion_df, x_column='LONGITUDE', y_column='LATITUDE')

# Convert the SEDF to a feature class without sanitizing columns where do i want this to live... its all of the gdb's combined so where do i update it.. to each gdb or individually or create a new one or put a copy into scratch.gdb and then some how write back 
sedf.spatial.to_featureclass(location=os.path.join(workspace, 'Erosion_Staging'), sanitize_columns=False)


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 18)

### Channel Incision

In [ ]:
#Channel Incision Process Data

# List of paths to geodatabases
channelincisiongdbs = ['channelincision20gdb', 'channelincision22gdb', 'channelincision23gdb']

# Initialize an empty list to store DataFrames
channeldfs = []

# Iterate over each geodatabase
for gdb in channelincisiongdbs:
    # List feature classes or tables in the geodatabase
    feature_classes = arcpy.ListFeatureClasses("*", "All", gdb)
    tables = arcpy.ListTables("*", "All", gdb)
    
    # Iterate over feature classes and tables
    for fc in feature_classes + tables:
        # Convert feature class or table to DataFrame
        df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(fc, '*'))
        dhanneldfs.append(df)

# Concatenate all DataFrames into a single DataFrame
erosion_df = pd.concat(channeldfs)

#Processing of Channel Incision Data

### Invasive Species

In [ ]:
#Invasive Species

### HEadcuts


In [ ]:
#Headcuts

### Bioassessment/ Biotic Integrity

In [ ]:
#Bioassessment scores- get all stream data into sde.Stream first... then look at Biotic Integrit Data Source to find which stream was used to evaluate each meadow.. this will help with percent of stream miles I believe

In [ ]:
#Conifer Encroachment

In [ ]:
#Aquatic Organism /Fish Passage

##### Code to make it a spatially enabled dataframe

In [ ]:
#code to conver to spatially enable geodataframe
# Create a new DataFrame Final_df with columns based on the field mapping
#df = df.rename(columns=field_mapping)

# Convert DataFrame to Spatially Enabled DataFrame
#sedf = GeoAccessor.from_xy(df, x_column='LONGITUDE', y_column='LATITUDE')

# Convert the SEDF to a feature class without sanitizing columns
#sedf.spatial.to_featureclass(location=os.path.join(workspace, 'PlanAreaNoise_23_Staging'), sanitize_columns=False)

### Code for Grading each parameter

In [ ]:
# Grading for each parameter add to final SDE file?
#Defining Grade for Channel Stability based on Erosiondf[percent_unstable]
def categorize_erosion(Percent_Unstable):
    if 0 <= value < 5:
        return 'A'
    elif 5 <= value < 20:
        return 'B'
    elif 20 <= value < 50:
        return 'C'
    else:
        return 'D'


#Define Grade for Incision based off of bankful height to bankful depth

def categorize_incision(bankful_ratio)
     if 0 <= bankful_ratio < 1.2:
        return 'A'
    elif 1.2 <= bankful_ratio < 1.6:
        return 'B'
    elif 1.6 <= bankful_ratio < 2.1:
        return 'C'
    else:
        return 'D'

#Define Grade for Headcut based off of headcut size---fix this
def categorize_headcut(value)
     if value = 0
        return 'A'
    elif 1.2 <= value < 1.6:
        return 'B'
    elif value =
        return 'C'
    else:
        return 'D'

Define Grade for Invasive based off of invasive
